# Verifications for the Zernike branch

## 0. Set up the environment

This notebook compares the Zernike branch and the master (v2.10) branch. These tests should take less than an hour to complete.

### 0.1 Clone and compile
To start, please checkout the Zernike branch and the master branch (__v2.10, not the latest master__) in two different folders. The following git commands clone these two branches into a folder `Zernike` and a folder `master`.

```
git clone -b Zernike https://github.com/PrincetonUniversity/SPEC.git Zernike
git clone -b v2.10 https://github.com/PrincetonUniversity/SPEC.git master
```

After cloning, please compile them seperately as normal.

### 0.2 Dependencies
Before running this notebook, one will need to set up the `py_spec` package. Please use the package in the Zernike branch, not the master branch. 

One will need `numpy`, `h5py` and `f90nml` packages. If you haven't installed them, please do so by

`pip install --user numpy h5py f90nml`

Or one can choose to use `conda` to manage the packages.

One can set up the `py_spec` package by the following two means:

1. Setting the environment before starting Jupyter Notebook

`export PYTHONPATH=$PYTHONPATH:/path/to/Zernike/Utilities/pythontools`

2. Setting the environment in this Notebook by copying the following commands into a new cell and running them before anything else in this notebook:

```python
import sys
sys.path.append('/path/to/Zernike/Utilities/pythontools')
```

Then we can import `py_spec` and relevant objects in it.

In [1]:
import sys
sys.path.append('/home/abaillod/SPEC/Utilities/pythontools')

In [2]:
import py_spec
from py_spec import SPEC
from py_spec.SPECNamelist import SPECNamelist
import os
import numpy as np

### 0.3 Specify the path to the input files and the path to the executables, for both v2.10 and the Zernike branch

__Please change the following paths to match your machine.__ Also set up the mpi command.

In [3]:
Zernike_home_folder='/home/abaillod/SPEC'
Zernike_executable='/home/abaillod/SPEC/xspec'

master_home_folder='/home/abaillod/development/code/master'
master_executable='/home/abaillod/development/code/master/xspec'

mpirun = 'mpirun'

### 0.4 Define a few helper functions that runs the cases and compares the results

In [4]:
def run_case(input_file_name, ncpus=1, LradZernike=None, Lradmaster=None, Mpol=None, Ntor=None, others=None):
    
    class Result:
        pass
    
    # For the Zernike branch

    # enter Zernike test directory
    os.chdir(Zernike_home_folder)
    # uses 3 cpus
    Zernike_xspec_command=mpirun + ' -np {:d} ' + Zernike_executable
    Zernike_xspec_command=Zernike_xspec_command.format(ncpus)

    # read the namelist
    Zernike_namelist = SPECNamelist(input_file_name)
    # compute the force gradient from scratch
    Zernike_namelist['globallist']['LreadGF']=False
    # do not update Bns initially
    Zernike_namelist['numericlist']['LautoinitBn'] = 0
    # do not generate Poincare plots
    Zernike_namelist['diagnosticslist']['nppts']=0
    # replace the Lrad
    if LradZernike is not None:
        Zernike_namelist['physicslist']['Lrad'][0] = LradZernike

    # For the v2.1 branch

    # enter master test directory
    os.chdir(master_home_folder)
    # uses 3 cpus
    master_xspec_command=mpirun + ' -np {:d} ' + master_executable
    master_xspec_command=master_xspec_command.format(ncpus)

    # read the namelist
    master_namelist = SPECNamelist(input_file_name)
    # compute the force gradient from scratch
    master_namelist['globallist']['LreadGF']=False
    # do not update Bns initially
    master_namelist['numericlist']['LautoinitBn'] = 0
    # do not generate Poincare plots
    master_namelist['diagnosticslist']['nppts']=0
    # replace the Lrad
    if Lradmaster is not None:
        master_namelist['physicslist']['Lrad'][0] = Lradmaster
        
        
    # replace Mpol or Ntor
    if Mpol is not None or Ntor is not None:
        if Ntor is None:
            Ntor = Zernike_namelist['physicslist']['Ntor']
        if Mpol is None:
            Mtor = Zernike_namelist['physicslist']['Ntor']
        Zernike_namelist.update_resolution(Mpol, Ntor)
        master_namelist.update_resolution(Mpol, Ntor)
        
    # replace other namelist items
    if others is not None:
        for item in others:
            Zernike_namelist[item[0]][item[1]]=item[2]
            master_namelist[item[0]][item[1]]=item[2]
        
    # run the cases
    os.chdir(Zernike_home_folder)
    Zernike_output = Zernike_namelist.run(Zernike_xspec_command, force=True)
    os.chdir(master_home_folder)
    master_output = master_namelist.run(master_xspec_command, force=True)

    # define the result object
    result = Result()
    result.Zernike_output = Zernike_output
    result.master_output = master_output
    
    return result

def check_interface(result, key='Rbc', rtol=1e-12):
    # check if Rbc matches
    Zernike_b=np.array(getattr(result.Zernike_output.output, key))
    master_b=np.array(getattr(result.master_output.output, key))
    
    b_diff = np.max(np.abs(Zernike_b - master_b)) 

    if (b_diff < rtol):
        print(key, " matches, diff", b_diff)
    else:
        print(key, " doesn't match, diff", b_diff)
        
def check_field_on_interfaces(result, key='Bp', rtol=1e-12):
    # check if the magnetic field in xyz matches on the interfaces
    Mvol = result.Zernike_output.output.Mvol
    
    diff_B_max = 0.0
    B_max = 0.0
    for lvol in range(Mvol):
        Zernike_B = np.array(getattr(result.Zernike_output.grid, key)[0])
        master_B = np.array(getattr(result.master_output.grid, key)[0])
        # the inner boundary
        if (lvol > 0):
            diff_B = np.abs(Zernike_B[:,0]-master_B[:,0])
            diff_B_max = np.max([diff_B_max, np.max(diff_B)])
            B_max = np.max([B_max, np.max(np.abs(Zernike_B[:,0]))])

        # the outer boundary
        diff_B = np.abs(Zernike_B[:,-1]-master_B[:,-1])
        diff_B_max = np.max([diff_B_max, np.max(diff_B)])
        B_max = np.max([B_max, np.max(np.abs(Zernike_B[:,-1]))])

    # evaluate error
    diff_B_max = diff_B_max 

    if (diff_B_max < rtol):
        print(key, " matches, diff", diff_B_max)
    else:
        print(key, " doesn't match, diff", diff_B_max)

## 1. Test Lconstraint=0,1,2 cases, fixed boundary
### 1.1 Five volume slab with Lconstraint=0
The example here uses Lrad=16. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.

_It takes approximately a few seconds to complete the runs._

In [8]:
input_file_name='/home/abaillod/SPEC/InputFiles/TestCases/G1V05L0Fi.001.sp'
result = run_case(input_file_name, ncpus=2)

check_interface(result, key='Rbc')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs unsuccessfully, check terminal output.
SPEC is running...
SPEC runs unsuccessfully, check terminal output.


AttributeError: 'NoneType' object has no attribute 'output'

### 1.2 Three volume slab with Lconstraint=2
The example here uses Lrad=16. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.
This case is used in ci.

_It takes approximately a few seconds to complete the runs._

In [11]:
input_file_name='InputFiles/TestCases/G1V03L2Fi.002.sp'
result = run_case(input_file_name, ncpus=2)

check_interface(result, key='Rbc')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 8.881784197001252e-16
Bp  matches, diff 1.0408340855860843e-17


### 1.3 32 volume cylinder with Lconstraint=1
The example here uses Lrad=12. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.
This case is used in ci.

_It takes approximately a few seconds to complete the runs._

In [12]:
input_file_name='ci/G2V32L1Fi/G2V32L1Fi.001.sp'
result = run_case(input_file_name, ncpus=2)

check_interface(result, key='Rbc')
check_field_on_interfaces(result, key='Bp')

Initial guess of the interface geometry ignored: line  102
Initial guess of the interface geometry ignored: line  102
SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 2.8449465006019636e-16
Bp  matches, diff 2.3314683517128287e-15


### 1.4 Two-volume rotating ellipse case with Lconstraint=1
We force it to have Lrad=16 in the first volume. Since the radial resolution is very high, they should give the same answer. The magnetic field on the interfaces should match.

_It takes approximately 5 mins to complete the runs. (10s for Zernike, 5 mins for master)_

In [13]:
input_file_name='InputFiles/TestCases/G3V02L1Fi.001.sp'
result = run_case(input_file_name, ncpus=2, LradZernike=16, Lradmaster=16)

check_interface(result, key='Rbc')
check_interface(result, key='Zbs')
check_field_on_interfaces(result, key='BR')
check_field_on_interfaces(result, key='BZ')
check_field_on_interfaces(result, key='Bp')

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 7.912559496503491e-13
Zbs  matches, diff 9.325873406851315e-13
BR  matches, diff 2.0873580641733724e-13
BZ  matches, diff 1.497031665298465e-13
Bp  matches, diff 2.8657631823136853e-14


### 1.5 Single-volume W7X OP1.1 fixed boundary

The magnetic field on the plasma boundary is compared, tolerance set to 1e-11 (further increase resolution will lead to ill-conditioning in the master branch)

_It takes approximately a minute to complete the runs. (5s for Zernike, 30s for master)_

In [14]:
input_file_name='InputFiles/TestCases/G3V01L0Fi.002.sp'
result = run_case(input_file_name, ncpus=1, LradZernike=32, Lradmaster=14, Mpol=8, Ntor=8)

check_field_on_interfaces(result, key='BR',rtol=1e-11)
check_field_on_interfaces(result, key='BZ',rtol=1e-11)
check_field_on_interfaces(result, key='Bp',rtol=1e-11)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
BR  matches, diff 4.036104783722294e-12
BZ  matches, diff 5.508166839307549e-12
Bp  matches, diff 2.2598589666245061e-13


## 2. Test Lconstraint=0,1,2 cases, free boundary
### 2.1 Three-volume free-boundary vacuum case without stellarator symmetry, FOCUS coil
Section 4.1 and Figure 1 in Hudson PPCF 2020 free-boundary SPEC paper

_It takes approximately 10 mins to complete the runs. (10s for Zernike, 10mins for master)_

In [15]:
input_file_name='ci/toroidal_freeboundary_vacuum/G3V02L0Fr.sp'
result = run_case(input_file_name, ncpus=3, LradZernike=24, Lradmaster=14)

check_interface(result, key='Rbc',rtol=1e-11)
check_interface(result, key='Zbs',rtol=1e-11)
check_interface(result, key='Rbs',rtol=1e-11)
check_interface(result, key='Zbc',rtol=1e-11)
check_field_on_interfaces(result, key='BR',rtol=1e-11)
check_field_on_interfaces(result, key='BZ',rtol=1e-11)
check_field_on_interfaces(result, key='Bp',rtol=1e-11)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 1.7195134205394424e-12
Zbs  matches, diff 3.4533487180965494e-13
Rbs  matches, diff 2.953089162094358e-13
Zbc  matches, diff 3.704216837777108e-13
BR  matches, diff 1.5695708621699112e-12
BZ  matches, diff 1.3319345626428003e-12
Bp  matches, diff 2.213118577287787e-12


### 2.2 Free-boundary case with Lconstraint=1, Nvol=8, provided by A. Baillod

In this case we have to allow lower error (1e-11) tolerance between the two branches since further increasing `Lrad` for the master branch will lead to ill-conditioning.

_It takes approximately 30s to complete the runs. (10s for Zernike, 20s for master)_

In [23]:
input_file_name='InputFiles/TestCases/G3V08L1Fr.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=3, LradZernike=22, Lradmaster=9, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-11)
check_interface(result, key='Zbs', rtol=1e-11)
check_field_on_interfaces(result, key='BR', rtol=1e-11)
check_field_on_interfaces(result, key='BZ', rtol=1e-11)
check_field_on_interfaces(result, key='Bp', rtol=1e-11)

SPEC is running...
SPEC runs unsuccessfully, check terminal output.
SPEC is running...
SPEC runs successfully.


AttributeError: 'NoneType' object has no attribute 'output'

### 2.3 VMEC-SPEC benchmark case, 8 volumes

Section 4.6, Figure 13 and 14 in Hudson PPCF 2020 free-boundary SPEC paper

In this case we reduce `Mpol` to 8 to avoid ill-conditioning. We do not compare to VMEC, but compare the result from the two branches. We have to allow lower error (1e-10) tolerance.

_It takes approximately 3 mins to complete the runs. (1 mins for Zernike, 2 mins for master)_

In [17]:
input_file_name='InputFiles/Verification/FreeBoundVMEC/Nv=008.L=12.M=32.n.sp.end'

replace_namelist = list()
replace_namelist.append(('globallist', 'gBntol', 1e-9))
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=3, LradZernike=16, Lradmaster=10, Mpol=8, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-10)
check_interface(result, key='Zbs', rtol=1e-10)
check_field_on_interfaces(result, key='BR', rtol=1e-10)
check_field_on_interfaces(result, key='BZ', rtol=1e-10)
check_field_on_interfaces(result, key='Bp', rtol=1e-10)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  doesn't match, diff 0.046951607953181096
Zbs  doesn't match, diff 0.040578280281378554
BR  doesn't match, diff 7.548983592228515e-06
BZ  doesn't match, diff 0.00010768775658123904
Bp  doesn't match, diff 0.0009112676722042232


## 3. Current constraint
### 3.1 Fixed boundary, Lconstraint=3, Lfindzero=1 test 

Test the Beltrami field and evaluation of the force with the current constraint. The force gradient (Lfindzero=2) is not used here

_This case takes approximately 2mins to run_



In [18]:

input_file_name = 'InputFiles/Verification/currentconstraint/TestCases_Comparison/G3V02L3Fi.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=2, LradZernike=16, Lradmaster=10, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-11)
check_interface(result, key='Zbs', rtol=1e-11)
check_field_on_interfaces(result, key='BR', rtol=1e-11)
check_field_on_interfaces(result, key='BZ', rtol=1e-11)
check_field_on_interfaces(result, key='Bp', rtol=1e-11)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  matches, diff 4.752587212664139e-13
Zbs  matches, diff 3.921229660419634e-13
BR  matches, diff 1.0839246167293481e-13
BZ  matches, diff 8.220507607958893e-14
Bp  matches, diff 1.3461454173580023e-14


## 3.2 Free boundary, Lconstraint=3, Lfindzero=1 test

Same as 3.1, but this time in free boundary

_This test is quite slow due to Lfindzero=1. Expect 30min to run_

In [19]:

input_file_name = 'InputFiles/Verification/currentconstraint/TestCases_Comparison/G3V08L3Fr.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=8, LradZernike=16, Lradmaster=10, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-11)
check_interface(result, key='Zbs', rtol=1e-11)
check_field_on_interfaces(result, key='BR', rtol=1e-11)
check_field_on_interfaces(result, key='BZ', rtol=1e-11)
check_field_on_interfaces(result, key='Bp', rtol=1e-11)

SPEC is running...
SPEC runs successfully.
SPEC is running...
SPEC runs successfully.
Rbc  doesn't match, diff 0.0014289624449670413
Zbs  doesn't match, diff 0.00019816370412784057
BR  doesn't match, diff 6.104892963350664e-07
BZ  doesn't match, diff 1.0979968569730866e-06
Bp  doesn't match, diff 3.3091531732365453e-05


## 3.3 Free boundary, Lconstraint=3, Lfindzero=2

Now that we are sure that the Beltrami field and the force evaluation is correct, test the force gradient using Lconstraint=2. 

In [11]:
input_file_name = '/home/abaillod/SPEC/InputFiles/TestCases/G3V08L3Fr.001.sp'

replace_namelist = list()
replace_namelist.append(('numericlist', 'Ndiscrete', 4))

result = run_case(input_file_name, ncpus=8, LradZernike=16, Lradmaster=6, others=replace_namelist)

check_interface(result, key='Rbc', rtol=1e-11)
check_interface(result, key='Zbs', rtol=1e-11)
check_field_on_interfaces(result, key='BR', rtol=1e-11)
check_field_on_interfaces(result, key='BZ', rtol=1e-11)
check_field_on_interfaces(result, key='Bp', rtol=1e-11)

SPEC is running...
SPEC runs unsuccessfully, check terminal output.
SPEC is running...
SPEC runs unsuccessfully, check terminal output.


AttributeError: 'NoneType' object has no attribute 'output'